In [3]:
import cv2
import numpy as np
import dotenv
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

dotenv.load_dotenv()


True

In [4]:
def create_model():
    '''Create CNN model for emotion detection'''

    model = Sequential([
        #First Conv BLock
        Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.25),


        #Second Conv BLock
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),
        Dropout(0.25),

        #Dense Layer
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(7, activation='softmax')
        ]) #7 emotions


        #Compile the model
    model.compile(
        optimizer = 'adam',
        loss = 'categorical_crossentropy',
        metrics = ['accuracy'])

    return model
        

In [ ]:
def train_model():
    #Set up data generators
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
        )

    #Setup directories
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    data_dir = os.path.join(parent_dir, 'data', 'model_datas')
    train_dir = os.path.join(data_dir, 'train')


    #Create data generators
    train_generators = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical',
        subset='training'
    )  

    validation_generators = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical',
        subset='validation'
    )

    #Create and compile model
    model = create_model()

    #Create checkpoint callback
    checkpoint_path = os.path.join('models', 'saved_models', 'emotion_model.keras')
    checkpoint_dir = os.path.dirname(checkpoint_path)
    os.makedirs(checkpoint_dir, exist_ok=True)

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )


    #Train the model
    history = model.fit(
        train_generators,
        validation_data=validation_generators,
        epochs=30,
        callbacks=[checkpoint_callback]
    )


    return model, history

In [24]:
if __name__ == '__main__':
    model,history = train_model()
    model.save('emotion_model.keras')

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Epoch 1/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.1963 - loss: 2.5620

/opt/anaconda3/envs/deepLearning/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


359/359 ━━━━━━━━━━━━━━━━━━━━ 58s 157ms/step - accuracy: 0.1963 - loss: 2.5612 - val_accuracy: 0.2069 - val_loss: 1.9659
Epoch 2/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 58s 161ms/step - accuracy: 0.2589 - loss: 1.8918 - val_accuracy: 0.2982 - val_loss: 1.7423
Epoch 3/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 108s 301ms/step - accuracy: 0.3196 - loss: 1.7244 - val_accuracy: 0.2578 - val_loss: 1.7958
Epoch 4/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 95s 265ms/step - accuracy: 0.3464 - loss: 1.6694 - val_accuracy: 0.3377 - val_loss: 1.6934
Epoch 5/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 96s 268ms/step - accuracy: 0.3647 - loss: 1.6081 - val_accuracy: 0.3954 - val_loss: 1.5780
Epoch 6/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 125s 350ms/step - accuracy: 0.4035 - loss: 1.5354 - val_accuracy: 0.4224 - val_loss: 1.4926
Epoch 7/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 110s 305ms/step - accuracy: 0.4315 - loss: 1.4767 - val_accuracy: 0.4227 - val_loss: 1.4449
Epoch 8/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 115s 320ms/step - accuracy: 0.4456 - loss: 1.4329 -